In [3]:
from pandas import read_csv
from torch.optim import SGD
from torch.nn import MSELoss
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter
from torch import randperm
from config import DATA_DIR
from src.data_set import RatingsDataset
from src.loss import MiningOutliersLoss
from src.model import MF
from src.runner import Runner
from src.utils import create_dataset, mine_outliers, DataConverter, DataProcessor, mean_centralised


"""
The Deam dataset is based on Arousal-Valence 2D emotional model.
The Valence/Arousal ratings were collected using Amazon Mechanical Turks service.
Each turk from the collected crowd were asked to mark his own emotion for the current song on a 2D plane, Arousal/Valence.
For more information please read: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0173392
"""

DF_PATH = f"{DATA_DIR}" \
          f"/DEAM/annotations/annotations per each rater/" \
          f"song_level/static_annotations_songs_1_2000.csv"

def select_n_random(trainset: RatingsDataset):
    """
    Selects n random data points and their corresponding labels from a dataset
    """
    perm = randperm(len(trainset))
    return trainset[perm][:100]

In [4]:
"""
This block of code calculates the outliers alongside the valence axis
"""
columns = ["workerID", "SongId", "Valence"]
original_df = read_csv(DF_PATH, skipinitialspace=True, usecols=columns)
original_df.columns = ["user_id", "item_id", "rating"]
original_df = mean_centralised(dataframe=original_df)

data_converter = DataConverter(
        original_df=original_df, n_random_users=10, n_ratings_per_random_user=9
)

valence_model = MF(
    n_users=data_converter.n_users,
    n_items=data_converter.n_item,
)
epochs = 50

criterion = MSELoss()
optimizer = SGD(valence_model.parameters(), lr=5, weight_decay=1e-3)
runner = Runner(
    model=valence_model,
    criterion=criterion,
    optimizer=optimizer,
    epochs=epochs
)

train_set = create_dataset(data_converter=data_converter)
train_load = DataLoader(train_set, batch_size=1000, shuffle=True)
users, items, ratings = select_n_random(train_set)

with SummaryWriter("runs/DEAM/valence") as writer:
    writer.add_graph(valence_model, (users, items))

    for epoch in range(epochs):
        epoch_loss = runner.train(train_loader=train_load, epoch=epoch, writer=writer)
        print(f"epoch={epoch + 1}, loss={epoch_loss}")

100%|██████████| 18/18 [00:01<00:00, 17.59batch/s, train_loss=0.00658]


epoch=1, loss=0.08053845308403676


100%|██████████| 18/18 [00:00<00:00, 35.10batch/s, train_loss=0.00632]


epoch=2, loss=0.06756571026829605


100%|██████████| 18/18 [00:00<00:00, 46.90batch/s, train_loss=0.0052] 


...

epoch=48, loss=0.008663185936449238


100%|██████████| 18/18 [00:00<00:00, 93.62batch/s, train_loss=0.000829]


epoch=49, loss=0.008515949558480122


100%|██████████| 18/18 [00:00<00:00, 88.51batch/s, train_loss=0.000734]

epoch=50, loss=0.008435969171218494


In [6]:
"""
This block of code calculates the outliers alongside the arousal axis
"""
columns = ["workerID", "SongId", "Arousal"]
original_df = read_csv(DF_PATH, skipinitialspace=True, usecols=columns)
original_df.columns = ["user_id", "item_id", "rating"]
original_df = mean_centralised(dataframe=original_df)

data_converter = DataConverter(
        original_df=original_df, n_random_users=10, n_ratings_per_random_user=9
)

arousal_model = MF(
    n_users=data_converter.n_users,
    n_items=data_converter.n_item,
)
epochs = 50

criterion = MSELoss()
optimizer = SGD(arousal_model.parameters(), lr=5, weight_decay=1e-3)
runner = Runner(
    model=arousal_model,
    criterion=criterion,
    optimizer=optimizer,
    epochs=epochs,
)

train_set = create_dataset(data_converter=data_converter)
train_load = DataLoader(train_set, batch_size=1000, shuffle=True)
users, items, ratings = select_n_random(train_set)

epochs = 50
with SummaryWriter("runs/DEAM/arousal") as writer:
    writer.add_graph(arousal_model, (users, items))

    for epoch in range(epochs):
        epoch_loss = runner.train(train_loader=train_load, epoch=epoch, writer=writer)
        print(f"epoch={epoch + 1}, loss={epoch_loss}")

100%|██████████| 18/18 [00:02<00:00,  7.17batch/s, train_loss=0.00701]


epoch=1, loss=0.08052921229493318


100%|██████████| 18/18 [00:00<00:00, 36.93batch/s, train_loss=0.00578]


epoch=2, loss=0.0665833346662969


100%|██████████| 18/18 [00:00<00:00, 30.34batch/s, train_loss=0.00438]


...

epoch=48, loss=0.008489833172585561


100%|██████████| 18/18 [00:00<00:00, 25.13batch/s, train_loss=0.00107] 


epoch=49, loss=0.008631662661525747


100%|██████████| 18/18 [00:00<00:00, 27.84batch/s, train_loss=0.000992]

epoch=50, loss=0.008527898675375466


In [7]:
valence_outliers = mine_outliers(model=valence_model, data_converter=data_converter)
arousal_outliers = mine_outliers(model=arousal_model, data_converter=data_converter)

items_group_by_users = data_converter.original_df.groupby("user_id")
combined_outliers = {}
for user_id, valence_dist in valence_outliers.items():
    arousal_dist = arousal_outliers[user_id]
    combined_outliers[user_id] = valence_dist + arousal_dist

combined_outliers = dict(sorted(combined_outliers.items(), key=lambda item: item[1]))
for user_id, item_id in combined_outliers.items():
    number_of_items = len(items_group_by_users.get_group(user_id))
    print(f"user: {user_id}, dist: {item_id}, #items: {number_of_items}")

user: ff18a27328ffd40ef52b7ebb7a0ded94, dist: -75.23977661132812, #items: 20
user: random_guy_194, dist: -65.95697021484375, #items: 9
user: random_guy_195, dist: -65.27352905273438, #items: 9
user: 19fee46f2810f34a8b69a7768d897a59, dist: -48.07280349731445, #items: 1
user: 2f790705ae66e70e81cc0f11ce0f4b9b, dist: -40.8505859375, #items: 2
user: random_guy_191, dist: -40.778480529785156, #items: 9
user: fd5b08ce362d855ca9152a894348130c, dist: -36.675235748291016, #items: 222
user: 3111e02887b600ee085c72c0a3df33e8, dist: -32.53719711303711, #items: 1
user: random_guy_190, dist: -28.196727752685547, #items: 9
user: random_guy_192, dist: -28.08419418334961, #items: 9
user: random_guy_193, dist: -26.771160125732422, #items: 9
user: e105c200f413d7b2c5850c0df4b9687e, dist: -22.180830001831055, #items: 2
user: 7cecbffe1da5ae974952db6c13695afe, dist: -22.118755340576172, #items: 428
user: b8ef6a913a63225faafd661ee2e1a7c0, dist: -15.897381782531738, #items: 10
user: random_guy_188, dist: -15.346